<a href="https://colab.research.google.com/github/chumpblocckami/AmazonTextClustering/blob/master/Traditional_ML_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import math
import numpy as np
from matplotlib.pyplot import cm
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

## 1. Clustering
Based on a n-dimensional matrix, we need to find cluster of similar data.

In [ ]:
data = [[random.randint(1,100) for x in range(0,2)] for x in range(0,100)]

class KMeans():
  def __init__(self, data, n_cluster=3, k=10):
    self.data = data
    self.n_clusters = n_cluster
    self.k = k
    self.centroids = self.init_centroids()
    self.labels = {}


  def init_centroids(self):
    "init centroids"
    dims = len(self.data[0])
    upper_limits = []
    lower_limits = []
    for dim in range(dims):
      dim_max = 0
      dim_min = 9**99
      for feature in data:
          if feature[dim] > dim_max:
            dim_max = feature[dim]
          if feature[dim] < dim_min:
            dim_min = feature[dim]
      upper_limits.append(dim_max)
      lower_limits.append(dim_min)
    assert len(upper_limits) == len(upper_limits)
    centroids = []
    for n in range(self.n_clusters):
        centroid = [random.randint(lower_limits[x],upper_limits[x]) for x in range(dims)]
        centroids.append(centroid)
    return centroids

  def prepare_data(self):
    "check data integrity"
    return self.data

  def create_centroids(self):
    "based on the data, create two centroids"

  def get_distance(self,a,b):
    "return distance between two points"
    a = (x for x in a)
    b = (x for x in b)
    distance = math.sqrt(sum( (a - b)**2 for a, b in zip(a,b)))
    return distance

  def get_best_fit(self):
    "based on n centroids, search best fit"
    #for every point, get the distances from specific cluster
    distances = []
    for point in data:
      _distances = []
      for centroid in self.centroids:
        distance_point_centroids = self.get_distance(point,centroid)
        _distances.append(distance_point_centroids) 
      distances.append(_distances)

    #based on distance, assign a point to a centroids
    self.labels = {}
    for n,distance in enumerate(distances):
      label = distance.index(min(distance))
      if str(label) not in self.labels.keys():
        self.labels[str(label)] = [data[n]]
      else:
        self.labels[str(label)].append(data[n]) 
    
    #update centroids of a given class with the mean
    for label in self.labels.keys():
      _data = self.labels[label]
      avg = [float(sum(col))/len(col) for col in zip(*_data)]
      self.centroids[int(label)] = avg
  
  def plot_clusters(self,k):
    "plot clusters"
    pca = PCA(n_components=2)
    _data = pca.fit_transform(self.data)
    _centroids = pca.fit_transform(self.centroids)

    plt.figure()
    plt.title(str(k) + " iteration")

    color=iter(cm.rainbow(np.linspace(0,1,self.n_clusters)))
    color = list(color)
    if self.labels == {}:
          colors = "gray"
    else:
          colors = []
          for point in self.data:
            for cluster in self.labels.keys():
              if point in self.labels[cluster]:
                colors.append(color[int(cluster)])
        
    plt.scatter([x[0] for x in _data], [y[1] for y in _data],c=colors)

    c_colors = [color[self.centroids.index(x)] for x in self.centroids]
    plt.scatter([x[0] for x in _centroids], [y[1] for y in _centroids],c=c_colors,marker="*",edgecolors="black",s=100)


  def iter(self,plot=True):
    "iterate clustering"
    for k in range(self.k):
      if plot:
        self.plot_clusters(k)
      self.get_best_fit()


  def fit(self):
    self.prepare_data()

  def transform(self):
    self.iter()

  def fit_transform(self):
    self.fit()
    self.transform()

kmeans = KMeans(data)
kmeans.fit_transform()

print(kmeans.labels)

2. KNeighbours

In [ ]:
import copy
from collections import Counter

class Kneighbours():
  def __init__(self,data,labels,k):
    self.data = data
    self.labels = labels
    self.k = k
    self.distances = {}
    self.prediction_space = {}
    self.predicted_labels = {}

  def get_distance(self,a,b):
    "return distance between two points"
    a = (x for x in a)
    b = (x for x in b)
    distance = math.sqrt(sum( (a - b)**2 for a, b in zip(a,b)))
    return distance
  
  def get_k_neighbours(self):
    for n,point in enumerate(data):
      min_distances = [self.get_distance(point,x) for x in data]
      distances = copy.deepcopy(min_distances)
      distances.sort()
      distances = distances[:self.k]
      self.distances[n] = {min_distances.index(dist):dist for dist in distances}
      self.prediction_space[n] = [self.labels[lbl] for lbl in self.distances[n].keys()]
    
  def predict(self,point):
    min_distances = [self.get_distance(point,x) for x in data]
    distances = copy.deepcopy(min_distances)
    distances.sort()
    distances = distances[:self.k]
    label_space = Counter([self.labels[min_distances.index(dist)] for dist in distances])
    prediction = {key:values/sum(label_space.values()) for key,values in label_space.items()}
    return prediction

  def iter(self,data):
    self.predicted_labels = {}
    if isinstance(data[0],list):
      predictions = []
      for n,point in enumerate(data):
        prediction = self.predict(point)
        predictions.append(prediction)
      self.predicted_labels.update({n:predictions})
    else:
      self.predicted_labels = self.predict(data)

  def fit(self):
    self.get_k_neighbours()

  def transform(self,data):
    self.iter(data)
  
data = [[1,1,1,1,1],
        [1,1,1,1,2],
        [1,1,1,10,1],
        [1,1,1,11,1]]
labels = [0,0,1,1]

KNN = Kneighbours(data,labels,k=3)
KNN.fit()
print(KNN.distances)

print("to predict:",[[1,1,1,1,1]])
preds = KNN.transform([[1,1,1,1,1],[2,2,2,2,2]])
print(KNN.predicted_labels)

{0: {0: 0.0, 1: 1.0, 2: 9.0}, 1: {1: 0.0, 0: 1.0, 2: 9.055385138137417}, 2: {2: 0.0, 3: 1.0, 0: 9.0}, 3: {3: 0.0, 2: 1.0, 0: 10.0}}
to predict: [[1, 1, 1, 1, 1]]
{1: [{0: 0.6666666666666666, 1: 0.3333333333333333}, {0: 0.6666666666666666, 1: 0.3333333333333333}]}


In [ ]:
ciao = [1,1,1,2,3,2]
from collections import Counter
{k:v/sum(Counter(ciao).values()) for k,v in Counter(ciao).items()}
#max(Counter(ciao), key=Counter(ciao).get)

{1: 0.5, 2: 0.3333333333333333, 3: 0.16666666666666666}

3. Decision Trees

In [ ]:
data = [[1,1,1,1,1],
        [1,1,1,1,2],
        [1,1,1,10,1],
        [1,1,1,10,1]]
labels = [0,0,1,1]

class DecisionTrees():
  def __init__(self, data,labels):
    self.data = data
    self.labels = labels

  def get_gini(self,groups,class):
    instances = float([sum(len(group)) for group in groups])
    gini = 0
    for group in groups:
      size = float(len(group))
      score = 0
      for _class in classes:
        p = [x[-1] for x in group].count(_class) / size
        score += p * p
      gini += (1 - score) * (size/instances)
   return gini